*Деление многочленов над полем ℤ/2 для вычисления CRC.*

In [37]:
from collections import defaultdict
class Z2Error(Exception):
    pass
class Z2:
    def __init__(self,n=0):
        if isinstance(n,int):
            self.value=int(n%2)
        elif isinstance(n,Z2):
            self.value = n.value
        else:
            raise Z2Error("Cannot create a Z2 from " + str(type(n)))
    def __str__(self):
        return str(self.value)
    def __eq__(self,other):
        #Сравнение по модулю - можно и с int-ами.
        if isinstance(other, Z2):
            return self.value==other.value
        if isinstance(other, int):
            return self.value==(other%2)
        raise Z2Error("Cannot compare a Z2 to "+str(type(other)))
    def __add__(self,other):
        if isinstance(other,int):
            other = Z2(other)
        if isinstance(other, Z2):
            return Z2(self.value+other.value)
        raise Z2Error("Cannot add a Z2 and "+str(type(other)))
    def __sub__(self,other):
        return self+(-other)
    def __mul__(self,other):
        if isinstance(other,int):
            other = Z2(other)
        if isinstance(other, Z2):
            return Z2(self.value*other.value)
        raise Z2Error("Cannot multiply a Z2 and "+str(type(other)))
    def __int__(self):
        return self.value
    def __pow__(self,other):
        #Возведение в неотрицательную степень(по умножению)
        if isinstance(other,int):
            res = self
            for i in range(1,other):
                res *= self
            return res
    def __neg__(self):
        #Обратное по сложению - само число
        return Z2(self.value)
class PolynomialZ2Error(Exception):
    pass
class PolynomialZ2:
    def __init__(self,n):
        self.coeffs = defaultdict(Z2)
        if isinstance(n,int) or isinstance(n,Z2):
            self.coeffs[0]=Z2(n)
            self.deg = 0
        elif isinstance(n,list) and len(n)>=1:
            for i in range(0,len(n)):
                self.coeffs[i]=Z2(n[i])
            self.deg = len(n)-1
        elif isinstance(n,dict):
            #Лучше бы этому словарю быть хорошим.
            self.coeffs = n
            self.deg = max(n.keys())
        else:
            raise PolynomialZ2Error("Cannot create a Z2 from " + str(type(n)))
    def __str__(self):
        res= '+'.join([f"{self.coeffs[i]}x^{i}" for i in self.coeffs.keys() if self.coeffs[i]!=0])
        if len(res)==0:
            return "0"
        return res
    def __eq__(self,other):
        #Сравнение - можно с int-ами и со списками.
        if isinstance(other, Z2) or isinstance(other, int) or isinstance(other, list):
            other = PolynomialZ2(other)
        if isinstance(other, PolynomialZ2):
            #Честно говоря, для Z/2 это бесполезно и жутко неэффективно(достаточно сравнить множества определённых коэффициентов),
            #но как задел на полиномы над любым кольцом...
            ok = True
            testCoeffs = set.union(self.coeffs.keys(),other.coeffs.keys())
            for i in testCoeffs:
                if self.coeffs[i]!=other.coeffs[i]:
                    ok=False
                    break
            return ok
        raise PolynomialZ2Error("Cannot compare a PolynomialZ2 to "+str(type(other)))
    def __add__(self,other):
        if isinstance(other, Z2) or isinstance(other, int) or isinstance(other, list):
            other = PolynomialZ2(other)
        if isinstance(other, PolynomialZ2):
            addCoeffs = set.union(set(self.coeffs.keys()),set(other.coeffs.keys()))
            c = dict()
            for i in addCoeffs:
                c[i]=self.coeffs[i]+other.coeffs[i]
            return PolynomialZ2(c)
        raise PolynomialZ2Error("Cannot add a PolynomialZ2Error and "+str(type(other)))
    def __sub__(self,other):
        return self+(-other)
    def __mul__(self,other):
        if isinstance(other, Z2) or isinstance(other, int) or isinstance(other, list):
            other = PolynomialZ2(other)
        if isinstance(other, PolynomialZ2):
            res = PolynomialZ2(0)
            for i in self.coeffs.keys():
                for j in other.coeffs.keys():
                    res.coeffs[i+j] += self.coeffs[i]*other.coeffs[j]
            res.deg = self.deg+other.deg #Наверно.
            return res
        raise PolynomialZ2("Cannot multiply a PolynomialZ2 and "+str(type(other)))
    def __int__(self):
        if(self.deg==0):
            return self.coeffs[0]
        raise PolynomialZ2("Cannot get an integer representation of a PolynomialZ2 that isn't a constant")
    def __pow__(self,other):
        #Возведение в неотрицательную степень(по умножению)
        if isinstance(other,int):
            res = self
            for i in range(1,other):
                res *= self
            return res
    def __neg__(self):
        c = {i:-self.coeffs[i] for i in self.coeffs.keys()}
        return PolynomialZ2(c)
a= PolynomialZ2([1,2,3])
print(a)
b = PolynomialZ2([0,1,0,1])
print(b*a)
print(-a)
print(a-a)
print(a+a)
print(a*2)
print(a*3)
print(a/2)

1x^0+1x^2
1x^1+1x^5
1x^0+1x^2
0
0
0
1x^0+1x^2


TypeError: unsupported operand type(s) for /: 'PolynomialZ2' and 'int'